### Проектный практикум: Классификация с использованием набора данных об академической успеваемости

ydata_profiling - быстрый анализ DA. Это первый этап разработки
from ydata_profiling import ProfileReport

Коэфицинеты: Пирсона, Спирмена, Кендала, ...

df.phik_matrix()


data leakage - учтечка данных (при расчете на полной выборке до разделения на трейн и тест)
Тест нельзя включать никогда !!! Иначе модель подсмотрит в правильные данные

коэф детерминации используется для оценки качества модели = сумм ошибок^2 / среднеее


**Установка пакетов**

In [25]:
import pandas as pd
import numpy as np

**1. Загрузка данных**

In [26]:
df_train = pd.read_csv('./data/train.csv')
df_train.set_index('id')
# df_test = pd.read_csv('./data/test.csv')
df_train

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,76513,1,17,1,9254,1,1,121.0,1,19,...,0,6,8,5,10.600000,0,13.9,-0.3,0.79,Graduate
76514,76514,1,1,6,9254,1,1,125.0,1,1,...,0,6,9,6,13.875000,0,9.4,-0.8,-3.12,Graduate
76515,76515,5,17,1,9085,1,1,138.0,1,37,...,0,5,8,5,11.400000,1,9.4,-0.8,-3.12,Enrolled
76516,76516,1,1,3,9070,1,1,136.0,1,38,...,0,6,0,0,0.000000,0,7.6,2.6,0.32,Dropout


**2. Анализ данных**
- выбор признаков
- масштабирование

In [27]:
!pip install ydata_profiling
from ydata_profiling import ProfileReport

profile = ProfileReport(df_train, title="Отчет по анализу")
profile

AttributeError: module 'numpy' has no attribute '_no_nep50_warning'

Ручной анализ

In [29]:
# df_train

head = df_train.head(3)
head

# info = df_train.info()
# print(info)

# describe = df_train.describe()
# describe
# print(describe)

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout


**Удаляет ненужные столбцы**

Ниже — **честный, профессиональный разбор**, какие столбцы оставить, какие удалить, и почему — если ты хочешь делать **регрессионный анализ** или **классификацию** (Target = Graduate / Dropout).

---

# ✅ 1. Столбцы, которые **точно нужно удалить**

Эти признаки **не несут прогностической ценности** или создают утечку данных → их нельзя использовать.

### ❌ 1. `id`

* Просто идентификатор строки.
* Не влияет на таргет.
* **Удаляем.**

---

# ❌ 2. Столбцы, создающие *утечку данных* (data leakage)

**Если цель — спрогнозировать статус студента заранее**, то некоторые признаки нельзя использовать, потому что они содержат *результаты обучения*, которые становятся известны только в конце.

Нельзя использовать **любые признаки, которые включают фактические оценки или число пройденных юнитов**, если цель — предсказать dropout заранее.

Это признаки:

### ❌ Curricular units (1st sem …)

* `Curricular units 1st sem (enrolled)`
* `Curricular units 1st sem (evaluations)`
* `Curricular units 1st sem (approved)`
* `Curricular units 1st sem (grade)`
* `Curricular units 1st sem (without evaluations)`
* то же самое для **2nd sem**.

Почему:

* Они напрямую показывают успехи в учебе.
* Модель будет "жульничать", зная оценки заранее.

Если ты хочешь моделировать реальный **прогноз Dropout на момент поступления**, то **удаляем все "Curricular units…"**.

Если же задача — **объясняющая регрессия после окончания**, тогда их можно оставить.

---

# ❌ 3. Макроэкономические показатели

* `Unemployment rate`
* `Inflation rate`
* `GDP`

Причина:

* Эти значения не меняются между студентами (в таблице у всех 11.1 / 0.6 / 2.02).
* Это **константы** → модель не получает пользы.
* **Удаляем.**

---

# ❌ 4. Категория TARGET

Если цель — регрессия, а Target — категория (`Graduate` / `Dropout`), то:

* либо надо менять задачу на *классификацию*,
* либо переводить таргет в 0/1.

---

# ✅ 2. Столбцы, которые **надо оставить**

Это демографические и входные данные:

* `Marital status`
* `Application mode`
* `Application order`
* `Course`
* `Daytime/evening attendance`
* `Previous qualification`
* `Previous qualification (grade)`
* `Nacionality`
* `Mother's qualification`
* `Father's qualification`
* `Mother's occupation`
* `Father's occupation`
* `Admission grade`
* `Age at enrollment`
* `Scholarship holder`
* `Debtor`
* `Tuition fees up to date`
* `Gender` (если есть в полном датасете)
* `Displaced`
* `Educational special needs`
* `Working student`

И другие похожие.

---

# ⚠️ 3. Зависит от цели: прогноз dropout (до начала семестра) или анализ завершивших обучение?

### Если цель → **Предсказание Dropout в самом начале обучения**

Удалить нужно много признаков:

* все "Curricular units"
* все оценки
* все факты, которые становятся известны позже

### Если цель → **Понять, что влияет на итоговый успех (ретроспектива)**

Можно оставить всё, кроме `id` и констант.

---

# 📌 Итоговое короткое решение

### Удалить:

* `id`
* Все `Curricular units ...`
* `Unemployment rate`
* `Inflation rate`
* `GDP`

Если делаем честный прогноз — удалить и `Previous qualification (grade)` и `Admission grade`, если они становятся известны только после экзамена при поступлении (зависит от твоих данных).

---

Если хочешь — могу:
✅ провести авто-анализ всех признаков
✅ составить точный список удаляемых колонок на основе полного датасета
✅ построить heatmap корреляций
✅ сделать baseline модель и показать, как влияет удаление признаков

Хочешь?


In [ ]:
df = df_train.drop(
    [
    'Nacionality',
    'Unemployment rate', 
    'Inflation rate', 'GDP'
    ],
    axis=1
)
df

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Mother's qualification,Father's qualification,...,Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Target
0,0,1,1,1,9238,1,1,126.0,1,19,...,6,14.500000,0,0,6,7,6,12.428571,0,Graduate
1,1,1,17,1,9238,1,1,125.0,19,19,...,4,11.600000,0,0,6,9,0,0.000000,0,Dropout
2,2,1,17,2,9254,1,1,137.0,3,19,...,0,0.000000,0,0,6,0,0,0.000000,0,Dropout
3,3,1,1,3,9500,1,1,131.0,19,3,...,7,12.591250,0,0,8,11,7,12.820000,0,Enrolled
4,4,1,1,2,9500,1,1,132.0,19,37,...,6,12.933333,0,0,7,12,6,12.933333,0,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,76513,1,17,1,9254,1,1,121.0,19,1,...,6,10.666667,0,0,6,8,5,10.600000,0,Graduate
76514,76514,1,1,6,9254,1,1,125.0,1,38,...,4,13.000000,0,0,6,9,6,13.875000,0,Graduate
76515,76515,5,17,1,9085,1,1,138.0,37,37,...,4,12.500000,2,0,5,8,5,11.400000,1,Enrolled
76516,76516,1,1,3,9070,1,1,136.0,38,37,...,0,0.000000,0,0,6,0,0,0.000000,0,Dropout
